In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
import os
os.chdir("../..")


In [6]:
import numpy as np

from numpy.typing import NDArray
from typing import Callable


from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator


In [7]:
from qml.model.gate import Gateset
from qml.model.unit import Unit, UnitManager
from qml.model.encoding import EncodingUnit, EncodingUnitManager
from qml.model.model import Model
from qml.tools.dataset import Dataset
from qml.tools.dataloader import DataLoader
from qml.tools.typing import Vector
from qml.tools.random import XRandomGenerator


## Parameters

In [8]:
nq = 2
ng = 3
nx = 1
ny = 1

# dataset
train_db_size = 10
validate_db_size = 10
# loader
batch_size = 4


In [9]:
target_func = lambda x: np.sin(2 * x)
rng = XRandomGenerator()


## Dev

In [36]:
class EvalResult:

    def __init__(self, xs: NDArray, ys: NDArray) -> None:
        self._xs = np.asarray(xs)
        self._ys = np.asarray(ys)
    
    @property
    def xs(self):
        return self._xs.copy()

    @property
    def ys(self):
        return self._ys.copy()


class Evaluator:

    def __init__(
            self,
            dataset: Dataset,
            model: Model = None,
            batch_size: int = None,
            shots: int = 50,
            seed: int = None,
            raise_iteration_error: bool = True,
    ):
        self._rng = XRandomGenerator(seed)
        self.dataset = dataset
        self.model = model
        self.batch_size = batch_size = batch_size if batch_size is not None else len(dataset)
        self.shuffle = False
        self.shots = shots

        self._loader = DataLoader.from_dataset(
            dataset, batch_size, shuffle=self.shuffle, seed=self._rng.new_seed()
        )
        self._loader_iter = None
        self._raise_iteration_error = raise_iteration_error
    
    def __call__(
            self,
            params: NDArray = None,
            model: Model = None
    ) -> EvalResult:
        if model is None:
            model = self.model
            
        if self._loader_iter is None:
            self._loader_iter = iter(self._loader)
        
        try:
            xs, ys = next(self._loader_iter)
        except StopIteration:
            if self._raise_iteration_error:
                raise StopIteration()
            self._loader_iter = iter(self._loader)
            xs, ys = next(self._loader_iter)
        
        return self.evaluate(
            model, params, xs, ys, shots=self.shots
        )
    
    @classmethod
    def evaluate(
        cls,
        model: Model,
        params: NDArray,
        xs: NDArray,
        ys: NDArray,
        shots: int = None,
    ):
        pass
        

In [38]:
class ErrorEvalResult(EvalResult):

    def __init__(self, errors: NDArray, xs: NDArray, ys: NDArray):
        super().__init__(xs, ys)
        self._es = np.asarray(errors)
    
    @property
    def es(self):
        return self._es.copy()
    
    @property
    def errors(self):
        return self.es


class ErrorEvaluator(Evaluator):

    def __init__(
            self,
            dataset: Dataset,
            model: Model = None,
            batch_size: int = None,
            shots: int = 50,
            seed: int = None
    ):
        super().__init__(dataset, model, batch_size, shots, seed)
    
    @classmethod
    def evaluate(
            cls,
            model: Model,
            params: NDArray,
            xs: NDArray,
            ys: NDArray,
            shots: int = None,
    ):
        ps = np.asarray([
            model.forward(x, params=params, shots=shots)
            for x in xs
        ])
        es = ys - ps
        res = ErrorEvalResult(es, xs, ys)
        return res


xs = rng.uniform(-1, 1, train_db_size)
ys = target_func(xs)
dataset = Dataset(xs, ys)

uman = UnitManager(nq, ng)
model = Model(
    nq, ny,
    EncodingUnitManager.AngleEncoding(nx, nq, repeat=True),
    uman.generate_random_unit(),
)

evl = ErrorEvaluator(dataset, model, 5)
res = evl()
print(res.es)
res = evl()
print(res.es)


[-1.37255812 -0.58267834  0.24407353  0.38630485 -1.15512685]
[ 0.65587139 -1.67636126  0.26893421  0.31957937  0.37514308]


In [39]:
class MSEEvalResult(EvalResult):

    def __init__(self, loss: float, xs: NDArray, ys: NDArray):
        super().__init__(xs, ys)
        self._loss = np.asarray(loss)
    
    @property
    def loss(self):
        return self._loss.copy()


class MSEEvaluator(Evaluator):

    def __init__(
            self,
            dataset: Dataset,
            model: Model = None,
            batch_size: int = None,
            shots: int = 50,
            seed: int = None
    ):
        super().__init__(dataset, model, batch_size, shots, seed)
    
    @classmethod
    def evaluate(
            cls,
            model: Model,
            params: NDArray,
            xs: NDArray,
            ys: NDArray,
            shots: int = None,
    ):
        res = ErrorEvaluator.evaluate(model, params, xs, ys, shots)
        loss = np.square(res.es).mean()
        return MSEEvalResult(loss, xs, ys)


xs = rng.uniform(-1, 1, train_db_size)
ys = target_func(xs)
dataset = Dataset(xs, ys)

uman = UnitManager(nq, ng)
model = Model(
    nq, ny,
    EncodingUnitManager.AngleEncoding(nx, nq, repeat=True),
    uman.generate_random_unit(),
)

evl = MSEEvaluator(dataset, model, 5)
res = evl()
print(res.loss)
res = evl()
print(res.loss)


0.6841242493248634
1.1560157476013377


In [13]:
class GradientEvalResult(ErrorEvalResult):

    def __init__(self, grads: NDArray, errors: NDArray, xs: NDArray, ys: NDArray) -> None:
        super().__init__(errors, xs, ys)
        self._grads = grads

    @property
    def gradients(self):
        return self._grads.copy()
    
    @property
    def grads(self):
        return self.gradients

    @property
    def loss(self):
        return np.square(self.es).mean()



class GradientEvaluator(Evaluator):

    demi_pi = np.pi / 2

    def __init__(
            self,
            dataset: Dataset,
            model: Model = None,
            batch_size: int = None,
            shots: int = 50,
            seed: int = None
    ):
        super().__init__(dataset, model, batch_size, shots, seed)
    
    @classmethod
    def evaluate(
            cls,
            model: Model,
            params: NDArray,
            xs: NDArray,
            ys: NDArray,
            shots: int = None,
    ):
        grads = np.asarray([
            cls.calc_gradient_idx_(model, params, x, shots=shots)
            for x in xs
        ])
        eres = ErrorEvaluator.evaluate(model, params, xs, ys, shots=shots)
        return GradientEvalResult(grads, eres.errors, xs, ys)
    

    @classmethod
    def calc_gradient_idx_(
            cls,
            model: Model,
            params: NDArray,
            x: NDArray,
            shots: int = None,
    ):
        if params is None:
            params = model.trainable_parameters
        trainable_params = params.copy()
        tp_shapes = [len(tp) for tp in trainable_params]
        tp_shapes.insert(0, 0)
        tp_shape_idxs = np.cumsum(tp_shapes)

        trainable_params = np.hstack(trainable_params)

        def deflatten(flattened):
            return [
                flattened[idx_de:idx_to]
                for idx_de, idx_to
                in zip(tp_shape_idxs[:-1], tp_shape_idxs[1:])
            ]
        
        def calc_gradient_idx(idx):
            shifted_pos = trainable_params.copy()
            shifted_neg = trainable_params.copy()
            shifted_pos[idx] = trainable_params[idx] + cls.demi_pi
            shifted_neg[idx] = trainable_params[idx] - cls.demi_pi

            predict_pos = model.forward(
                x,
                params=deflatten(shifted_pos),
                shots=shots,
            )
            predict_neg = model.forward(
                x,
                params=deflatten(shifted_neg),
                shots=shots,
            )
            grad = (predict_pos - predict_neg) / 2
            return grad
        
        grads = np.asarray([
            calc_gradient_idx(idx)
            for idx in range(len(trainable_params))
        ])
        return grads


In [41]:
xs = rng.uniform(-1, 1, train_db_size)
ys = target_func(xs)
dataset = Dataset(xs, ys)

uman = UnitManager(nq, ng)
model = Model(
    nq, ny,
    EncodingUnitManager.AngleEncoding(nx, nq, repeat=True),
    uman.generate_random_unit(),
    uman.generate_random_unit(),
)


In [42]:
evfunc = GradientEvaluator(dataset, model, 5)
res = evfunc()
print(res.gradients.shape)
res = evfunc()
print(res.gradients.shape)


(10, 3)
(10, 3)


In [18]:
class Wavelet:

    def __init__(self):
        pass

    def get_pattern_applied_func(self, a: float, b: float) -> Callable:
        pass

    def calc_wavelet_params(self, dim):
        return sum([
            [
                (a, b)
                for b in np.arange(-1, 1, a)
            ]
            for a in 2 / 2 ** np.arange(dim)
        ], [])
    
    @staticmethod
    def get_wavelet_range(wavelet_param):
        a, b = wavelet_param
        return [b, a + b]


class Haar(Wavelet):

    def __init__(self):
        pass

    def get_pattern_applied_func(self, a: float, b: float, xs) -> Callable:
        shifted_xs = (xs - b) / a
        yneg = np.where((-1 <= shifted_xs) & (shifted_xs < 0), -1, 0)
        ypos = np.where((0 <= shifted_xs) & (shifted_xs < 1), 1, 0)
        return (yneg + ypos) / np.sqrt(a)


class WaveletTransform:

    def __init__(self, wavelet: Wavelet) -> None:
        self._wavelet = wavelet
    
    def transform(self, xs, ys, dim=4):
        xs = np.asarray(xs)
        ys = np.asarray(ys)
        wparams = self.generate_wavelet_params(dim)
        powers = np.asarray([
            np.mean(
                self._wavelet.get_pattern_applied_func(*wparam, xs) * ys
            )
            for wparam in wparams
        ])
        return powers

    @staticmethod
    def generate_wavelet_params(dim):
        return np.asarray(sum([
            [
                (a, b)
                for b in np.arange(-1, 1, a)
            ]
            for a in 2 / 2 ** np.arange(dim)
        ], []))


In [44]:
class WaveletEvalResult(EvalResult):

    def __init__(self, errors: NDArray, powers: NDArray, xs: NDArray, ys: NDArray):
        super().__init__(xs, ys)
        self._es = np.asarray(errors)
        self._ps = np.asarray(powers)
    
    @property
    def es(self):
        return self._es.copy()
    
    @property
    def errors(self):
        return self.es
    
    @property
    def ps(self):
        return self._ps.copy()
    
    @property
    def powers(self):
        return self.ps


class WaveletEvaluator(Evaluator):

    def __init__(
            self,
            wavelet: Wavelet,
            dataset: Dataset,
            model: Model = None,
            wavelet_dim: int = 4,
            batch_size: int = None,
            shots: int = 50,
            seed: int = None
    ):
        super().__init__(dataset, model, batch_size, shots, seed)
        self._wavelet = wavelet
        self._wtrans = WaveletTransform(wavelet)
        self._wdim = wavelet_dim
    
    def __call__(
            self,
            params: NDArray = None,
            model: Model = None
    ) -> EvalResult:
        if model is None:
            model = self.model
        return self.evaluate(
            self._wtrans, self._wdim, model, params, self.dataset.xs, self.dataset.ys, shots=self.shots
        )

    @classmethod
    def evaluate(
            cls,
            wtrans: WaveletTransform,
            wdim: int,
            model: Model,
            params: NDArray,
            xs: NDArray,
            ys: NDArray,
            shots: int = None,
    ):
        eres = ErrorEvaluator.evaluate(model, params, xs, ys, shots=shots)
        powers = wtrans.transform(xs, eres.errors, dim=wdim)
        return WaveletEvalResult(eres.errors, powers, xs, ys)


eval = WaveletEvaluator(Haar(), dataset, model, batch_size=5)
res = eval()
print(res.powers)
res = eval()
print(res.powers)


[-0.40126696 -0.61804238  0.66860759 -0.51635187  0.15865983  0.29343329
  0.2000275  -0.35872607 -0.01277968  0.06935184  0.09845489  0.20369903
 -0.0908756   0.14077148  0.09221423]
[-0.37581112 -0.57804238  0.62460759 -0.47675389  0.13603241  0.28211959
  0.18305694 -0.31872607 -0.03677968  0.07735184  0.07445489  0.20369903
 -0.0828756   0.13277148  0.07621423]
